# Trova Domande Duplicate


## Word embedding

Two different models of embeddings:

 - [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. Download it by following this [link](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing).
 - Representations using StarSpace on StackOverflow data sample. 

It's always easier to start with pretrained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. 

In [0]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 50.2MB/s 
  Found existing installation: gensim 3.8.0
    Uninstalling gensim-3.8.0:
      Successfully uninstalled gensim-3.8.0


In [0]:
import gensim

In [0]:
wv_embeddings =gensim.models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000) 


### check if the loaded embeddings contain a word:
    
  

In [0]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [0]:
print(check_embeddings(wv_embeddings))

These embeddings look good.


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [0]:
import numpy as np

In [0]:
def question_to_vec(question, embeddings, dim=300):
    rslt = np.zeros((dim,), dtype=np.float32)
    k = 0
    for i in question.split():
        if i in embeddings:
            k += 1
            rslt += embeddings[i]
    if k == 0:
        resultat=rslt
    else:
        resultat= rslt/k
    
    return resultat
             

In [0]:
import nltk
nltk.download('stopwords')
from util import array_to_string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!




## Evaluation of text similarity

 
### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, and topK($q_i$) is the top of the ranked sentences provided by our model.


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ DCG = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

**HitsCount and DCGScore**  

In [0]:
def hits_count(best_ranks, k):
    mat=np.array([k])
    best_mat=np.array(best_ranks)
    return np.average(best_mat <= mat)

In [0]:
def dcg_score(dup_ranks, k):

    return np.average((np.array(dup_ranks) <= np.array([k]))*1./(np.log2(1. + np.array(dup_ranks))))


##  Pre-trained embeddings

In [0]:
validation =upload_corpus('/content/data/validation.tsv') 

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

Using cosine distance to rank candidate questions which I need to implement in the function *rank_questions*. The function should return a sorted list of pairs *(initial position in candidates list, question)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, than *a* and *b*, the functions should return a list *[(2, c), (0, a), (1, b)]*.


In [0]:
def rank_candidates(question, candidates, embeddings, dim=300):
    a = question_to_vec(question, embeddings, dim)[np.newaxis, :]
    b = np.array([question_to_vec(c, embeddings, dim) for c in candidates])
    resulat=[(e, candidates[e]) for e in np.argsort(cosine_similarity(a, b)[0])[::-1]]
    return resulat

Test the quality of the current approach.  

In [0]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [0]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("nDCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

nDCG@   1: 0.209 | Hits@   1: 0.209
nDCG@   5: 0.263 | Hits@   5: 0.311
nDCG@  10: 0.279 | Hits@  10: 0.360
nDCG@ 100: 0.316 | Hits@ 100: 0.548
nDCG@ 500: 0.349 | Hits@ 500: 0.807
nDCG@1000: 0.369 | Hits@1000: 1.000


Let's try to understand why the quality is so low, idea how the data looks like. Print several questions from the data:

In [0]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities


It means that I have many punctiation marks, special characters and unlowercased letters. In our case, it could lead to the situation where I can't find some embeddings, e.g. for the word "grid?". 

To solve this problem ,  functions *text_prepare*  to prepare the data.

In [0]:
from util import text_prepare

Now transform all the questions from the validation set:

In [0]:
prepared_validation = []
for line in validation:
    prepared_validation.append([text_prepare(texts) for texts in line])

Let's evaluate the approach again after the tokenization:

In [0]:
wv_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [0]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("nDCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                               k, hits_count(wv_prepared_ranking, k)))

nDCG@   1: 0.305 | Hits@   1: 0.305
nDCG@   5: 0.375 | Hits@   5: 0.438
nDCG@  10: 0.392 | Hits@  10: 0.489
nDCG@ 100: 0.425 | Hits@ 100: 0.656
nDCG@ 500: 0.447 | Hits@ 500: 0.830
nDCG@1000: 0.465 | Hits@1000: 1.000


Now, tokenize also train and test data:

In [0]:
def tokenize_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [0]:
tokenize_file('/content/data/train.tsv', '/content/data/train_toknize.tsv')
tokenize_file('/content/data/test.tsv', '/content/data/test_toknize.tsv')

In [0]:
from util import matrix_to_string
tokenized_test_data = '/content/data/test_toknize.tsv'

## StarSpace embeddings

 
The main point in this section is that StarSpace can be trained specifically for some tasks. but  word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases I get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 



In [0]:
#TRAINING

!Starspace/starspace train -trainFile '/content/data/train_toknize.tsv' -model "train_rslt" -trainMode 3 \
    -adagrad true -ngrams 1 -epoch 5 -dim 100 -minCount 2 -verbose true -fileFormat "labelDoc" \
    -negSearchLimit 10 -similarity "cosine" -lr 0.05

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
batchSize: 5
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
useWeight: 0
weightSep: :
Start to initialize starspace model.
Build dict from input file : /content/data/train_toknize.tsv
Read 12M words
Number of words in dictionary:  95058
Number of labels in dictionary: 0
Loading data from file : /content/data/train_toknize.tsv
Total number of examples loaded : 999740
Initialized model weights. Model size :
matrix : 95058 100
Training epoch 0: 0.05 0.01
Epoch: 100.0%  lr: 0.040010  loss: 0.043476  eta: 0h11m  tot: 0h2m52s  (20.0%)
 ---+++                Epoch    0 Train error : 0.04366956 +++--- ☃
Training epoch 1: 0.04 0.01
Epoch: 100.0%  lr: 0.030000  loss: 0.013185

Compare the new embeddings with the previous ones.  

In [0]:
starspace_embeddings = {} 
for line in open('/content/train_rslt.tsv'):
    valu = line.strip().split('\t')
    starspace_embeddings[valu[0]] = np.array(valu[1:], dtype=np.float32)

In [0]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [0]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("nDCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

nDCG@   1: 0.514 | Hits@   1: 0.514
nDCG@   5: 0.612 | Hits@   5: 0.696
nDCG@  10: 0.631 | Hits@  10: 0.756
nDCG@ 100: 0.663 | Hits@ 100: 0.907
nDCG@ 500: 0.672 | Hits@ 500: 0.980
nDCG@1000: 0.674 | Hits@1000: 1.000
